In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pyCloudy as pc
print(pc.__version__)

In [ ]:
# Tell pyCloudy where the executable is to be found
pc.config.cloudy_exe = '/usr/local/Cloudy/c17.02/source/cloudy.exe'

In [ ]:
# Define the directory where the models will be run
import os
dir_ = os.environ['HOME'] + '/SIGNALS_models'
pc.print_make_file(dir_)

In [ ]:
# The following function write N files corresponding to different angles
# of an elliptical PN
from pyCloudy.utils.physics import abund_Nicholls_GC_2017

def set_models(model_name, Teff, QH, a, dist, N_models):
    """
    model_name: generic name for the models
    N: number of models
    Teff: effective temperature of the central BB
    QH: number of ionizing photons of the central BB
    a: ellipsicity
    """
    emis_tab = ['H  1 4861.33A', 
                'H  1 6562.81A', 
                'N  2 6583.45A', 
                'O  2 3726.03A',
                'O  2 3728.81A',                          
                'O  3 5006.84A',
                'BLND 4363.00A',
                'S  2 6730.82A',
                'S  2 6716.44A']  
    thetas = np.linspace(0., 90., N_models)
    thetas_rad = np.pi / 180. * thetas
    # The fact_elli is used for the inner radius (increasing) and density (decreasing)
    fact_elli = a / np.sqrt((np.sin(thetas_rad))**2 + (a * np.cos(thetas_rad))**2)
    rs_in = 16.5 + np.log10(fact_elli)
    densities = 4 - np.log10(fact_elli) * 2
    
    model = pc.CloudyInput()
    model.set_BB(Teff, 'q(H)', QH)
    model.set_abund(ab_dict=abund_Nicholls_GC_2017)
    model.set_grains('ism')
    model.set_emis_tab(emis_tab)
    model.set_distance(dist=dist, unit='kpc', linear=True)
    
    for theta, r_in, density in zip(thetas, rs_in, densities):
        model.model_name = '{0}/{1}_{2:.0f}'.format(dir_, model_name,theta)
        model.set_cste_density(density)
        model.set_radius(r_in)
        model.set_theta_phi(theta)
        model.print_input(to_file = True, verbose = False)

In [ ]:
model_name = "M3D_1"
pc.log_.level = 3

In [ ]:
Teff = 40000
QH = 48.7
a = 2.
dist = 5
N_models = 6
set_models(model_name, Teff, QH, a, dist, N_models)

In [ ]:
pc.run_cloudy(dir_ = dir_, n_proc = 6, model_name = model_name, use_make = True)

In [ ]:
pc.log_.level = 3
list_of_models = pc.load_models('{0}/{1}'.format(dir_, model_name), 
                                 list_elem=['H', 'He', 'C', 'N', 'O', 'Ar', 'Ne'],  
                                 read_cont = False, read_grains = False)

In [ ]:
dim = 101

In [ ]:
m3d = pc.C3D(list_of_models, dims = [dim, dim, dim], angles = [45,45,0], plan_sym = True)

In [ ]:
def some_plots(m3d, proj_axis, n_cut):
    f, axes = plt.subplots(3,3, figsize=(15, 14))
    
    i_ax = 0
    ax = axes.ravel()[i_ax]
    im = ax.imshow(m3d.get_emis('H__1_486133A').sum(axis = proj_axis)*
               m3d.cub_coord.cell_size)
    ax.set_title('Hb')
    f.colorbar(im, ax=ax)
    
    i_ax += 1
    ax = axes.ravel()[i_ax]
    im = ax.imshow(m3d.get_emis('N__2_658345A').sum(axis = proj_axis)*
               m3d.cub_coord.cell_size)
    ax.set_title('[NII]')
    f.colorbar(im, ax=ax)
    
    i_ax += 1
    ax = axes.ravel()[i_ax]
    im = ax.imshow(m3d.get_emis('O__3_500684A').sum(axis = proj_axis)*
               m3d.cub_coord.cell_size)
    ax.set_title('[OIII]')
    f.colorbar(im, ax=ax)
    
    i_ax += 1
    ax = axes.ravel()[i_ax]
    im = ax.imshow(m3d.get_emis('N__2_658345A').sum(axis = proj_axis)/
               m3d.get_emis('H__1_486133A').sum(axis = proj_axis))
    ax.set_title('[NII]/Hb')
    f.colorbar(im, ax=ax)
    
    i_ax += 1
    ax = axes.ravel()[i_ax]
    ax.imshow(m3d.get_emis('O__3_500684A').sum(axis = proj_axis)/
               m3d.get_emis('H__1_486133A').sum(axis = proj_axis))
    ax.set_title('[OIII]/Hb')
    f.colorbar(im, ax=ax)
    
    i_ax += 1
    ax = axes.ravel()[i_ax]
    im = ax.imshow(m3d.get_ionic('O',1)[n_cut,:,:])
    ax.set_title('O+ cut')
    f.colorbar(im, ax=ax)
    
    i_ax += 1
    ax = axes.ravel()[i_ax]
    sc = ax.scatter(m3d.get_ionic('O',1).ravel(),
                np.log10(m3d.get_ionic('N',1).ravel()/
                         m3d.get_ionic('O',1).ravel()),
                c=np.abs(m3d.cub_coord.theta.ravel()), edgecolors = 'none')
    ax.set_title('Colored by |Theta|')
    ax.set_xlabel('O+ / O')
    ax.set_ylabel('log N+/O+ / N/O')
    f.colorbar(sc, ax=ax)
    
    i_ax += 1
    ax = axes.ravel()[i_ax]
    sc = ax.scatter(m3d.get_ionic('O',1).ravel(),
                np.log10(m3d.get_ionic('N',1).ravel()/
                         m3d.get_ionic('O',1).ravel()),
                c=m3d.relative_depth.ravel(),vmin = 0, vmax = 1, edgecolors = 'none')
    ax.set_title('Colored by radial angular position in the nebula')
    ax.set_xlabel('O+ / O')
    ax.set_ylabel('N+/O+ / N/O')
    f.colorbar(sc, ax=ax)
    
    i_ax += 1
    ax = axes.ravel()[i_ax]
    im = m3d.get_RGB(['N__2_658345A', 'H__1_486133A', 'O__3_500684A'], axes = proj_axis)
    ax.imshow(im)

In [ ]:
m3d.angles = [0,45,0]
some_plots(m3d, proj_axis, n_cut = int((dim-1) /2))

In [ ]:
# A function in form of lambda to transform size in cm into arcsec, for a distance "dist" defined above.
from pyCloudy.utils.astro import conv_arc
arcsec = lambda cm: conv_arc(dist=dist, dist_proj=cm)

In [ ]:
def make_mask(m3d, ap_center=[0., 0.], ap_size=[1., 1.]):
    """
    This returns a mask (values between 0. and 1.) to be multiplied to the image to take the flux passing through an aperture.
    An pc.C3D object named M_sphere must exist outside theis function
    """
    x_arc = arcsec(m3d.cub_coord.x_vec)
    y_arc = arcsec(m3d.cub_coord.y_vec)
    z_arc = arcsec(m3d.cub_coord.z_vec)
    X, Y = np.meshgrid(y_arc, x_arc)
    bool_mask = ((X > ap_center[0] - ap_size[0]/2.) & 
            (X <= ap_center[0] + ap_size[0]/2.) & 
            (Y > ap_center[1] - ap_size[1]/2.) & 
            (Y <= ap_center[1] + ap_size[1]/2.))
    mask = np.zeros_like(X)
    mask[bool_mask] = 1.0
    return mask

In [ ]:
# we define the mask. Can be change to see the effect of the aperture on line intensities
mask = make_mask(m3d, ap_center=[0.0, 1.0], ap_size=[50, 0.5])

In [ ]:
# We plot the OIII image and overplot the mask.
f, ax = plt.subplots()
im = ax.imshow(m3d.get_emis('O__3_500684A').sum(axis=1))
f.colorbar(im, ax=ax)
ax.contour(mask);

In [ ]:
# Hbeta is computed for the whole object and throught the aperture
Hb_tot = (m3d.get_emis('H__1_486133A') * m3d.cub_coord.cell_size).sum()
Hb_slit = ((m3d.get_emis('H__1_486133A') * m3d.cub_coord.cell_size).sum(1) * mask).sum()
print(Hb_tot, Hb_slit)

In [ ]:
# For every line, we compute the intensity for the whole object and throught the aperture.
# We also print out the difference due to the slit.
for label in m3d.m[0].emis_labels:
    I_tot = (m3d.get_emis(label).sum()*m3d.cub_coord.cell_size) / Hb_tot
    I_slit = ((m3d.get_emis(label).sum(1) * mask).sum()*m3d.cub_coord.cell_size) / Hb_slit
    print('line: {0:12s} I/Ib Total: {1:6.4f} I/Ib Slit: {2:6.4f} Delta: {3:4.1f}%'.format(label, I_tot, I_slit, 
                                                                                           (I_slit-I_tot)/I_tot*100))